In [ ]:
import numpy as np
import json
from collections import defaultdict, Counter
from scipy import spatial, stats
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from adjustText import adjust_text

In [ ]:
ROOT = '/mnt/data0/lucy/manosphere/'
DATA = ROOT + 'data/'
GLOVE = DATA + 'glove/'
LOGS = ROOT + 'logs/'
AGG_EMBED_PATH = LOGS + 'semantics_mano/agg_embed/'

## Inspecting individual words

In [ ]:
with open(LOGS + 'semantics_mano/results/scores.json', 'r') as infile: 
    scores = json.load(infile) 

vocab_order = []
with open(LOGS + 'semantics_mano/results/vocab_order.txt', 'r') as infile:
    vocab_order = infile.readlines()

with open(LOGS + 'coref_results/mano_gender_labels.json', 'r') as infile: 
    gender_labels = json.load(infile)

In [ ]:
word_poles = defaultdict(Counter) # {word : {pole : score}}
for pole in scores: 
    s = scores[pole]
    for i, term in enumerate(vocab_order): 
        term = term.strip()
        word_poles[term][pole] = s[i]

In [ ]:
print(word_poles['alpha'].most_common()[:10])
print(word_poles['alpha'].most_common()[-10:])

In [ ]:
print(word_poles['beta'].most_common()[:10])
print(word_poles['beta'].most_common()[-10:])

In [ ]:
print(word_poles['escort'].most_common()[:10])
print(word_poles['escort'].most_common()[-10:])

In [ ]:
print(word_poles['wife'].most_common()[:10])
print(word_poles['wife'].most_common()[-10:])

In [ ]:
print(word_poles['celebrities'].most_common()[:10])
print(word_poles['celebrities'].most_common()[-10:])

In [ ]:
print(word_poles['princes'].most_common()[:10])
print(word_poles['princes'].most_common()[-10:])

In [ ]:
print(word_poles['normies'].most_common()[:10])
print(word_poles['normies'].most_common()[-10:])

In [ ]:
print(word_poles['simps'].most_common()[:10])
print(word_poles['simps'].most_common()[-10:])

In [ ]:
print(word_poles['prostitute'].most_common()[:10])
print(word_poles['prostitute'].most_common()[-10:])

In [ ]:
print(word_poles['sweet girl'].most_common()[:10])
print(word_poles['sweet girl'].most_common()[-10:])

# Variance within gender

In [ ]:
fem_variance = Counter() # {pole: variance of fem terms}
fem_extremes = defaultdict(tuple) # {pole: (top N terms, bottom N terms)}
masc_variance = Counter() 
masc_extremes = defaultdict(tuple)
fem_scores = defaultdict(list) # {pole: scores in order of x_words}
masc_scores = defaultdict(list) # {pole: scores in order of y_words}
N = 10
for pole in scores: 
    s = scores[pole]
    x = [] # fem scores
    x_words = []
    y = [] # masc scores
    y_words = []
    for i, term in enumerate(vocab_order): 
        term = term.strip()
        if term in gender_labels: 
            if gender_labels[term] > 0.75: 
                x.append(s[i])
                x_words.append(term)
            elif gender_labels[term] < 0.25: 
                y.append(s[i])
                y_words.append(term)
    fem_variance[pole] = np.var(x)
    indices = np.argpartition(x, -N)[-N:]
    topN = [x_words[idx].strip() for idx in indices]
    indices = np.argpartition(x, N)[:N]
    bottomN = [x_words[idx].strip() for idx in indices]
    fem_extremes[pole] = (topN, bottomN, min(x), max(x))
    fem_scores[pole] = x
    
    masc_variance[pole] = np.var(y)
    indices = np.argpartition(y, -N)[-N:]
    topN = [y_words[idx].strip() for idx in indices]
    indices = np.argpartition(y, N)[:N]
    bottomN = [y_words[idx].strip() for idx in indices]
    masc_extremes[pole] = (topN, bottomN, min(y), max(y))
    masc_scores[pole] = y

In [ ]:
for tup in fem_variance.most_common(20): 
    print(tup)
    pole = tup[0]
    topN, bottomN, mi, ma = fem_extremes[pole]
    print("TOP:", topN)
    print("BOTTOM:", bottomN)
    print(mi, ma)
    print()

In [ ]:
for tup in masc_variance.most_common(20): 
    print(tup)
    pole = tup[0]
    topN, bottomN, mi, ma = masc_extremes[pole]
    print("TOP:", topN)
    print("BOTTOM:", bottomN)
    print(mi, ma)
    print()

## Axes similarities for each clusters' highly gendered terms

In [ ]:
with open(LOGS + 'gram_counts/combined_catyear_word_count.json', 'r') as infile: 
    catyear_word_count = json.load(infile)
total_vocab_count = Counter() # {term : count in mano reddit + forum}
for catyear in catyear_word_count: 
    for term in catyear_word_count[catyear]: 
        total_vocab_count[term] += catyear_word_count[catyear][term]
print(total_vocab_count.most_common(20))

In [ ]:
with open(LOGS + 'time_series/cluster_members_6.json', 'r') as infile: 
    clust_words = json.load(infile)
clust_words_rev = {}
for clust in clust_words: 
    cluster_words = clust_words[clust]
    for w in cluster_words: 
        clust_words_rev[w] = clust

In [ ]:
cluster_gender_sum = defaultdict(Counter) # {gender + clust_num : {pole : sum of word scores} }
cluster_gender_total = defaultdict(Counter) # {gender + clust_num : {pole : word count} }
for pole in scores: 
    s = scores[pole]
    for i, term in enumerate(vocab_order): 
        term = term.strip()
        clust_num = clust_words_rev[term]
        if term in gender_labels: 
            if gender_labels[term] > 0.75: 
                cluster_gender_sum[str(clust_num) + '_fem'][pole] += s[i]
                cluster_gender_total[str(clust_num) + '_fem'][pole] += 1
            elif gender_labels[term] < 0.25: 
                cluster_gender_sum[str(clust_num) + '_masc'][pole] += s[i]
                cluster_gender_total[str(clust_num) + '_masc'][pole] += 1
            else: 
                cluster_gender_sum[str(clust_num) + '_other'][pole] += s[i]
                cluster_gender_total[str(clust_num) + '_other'][pole] += 1

In [ ]:
cluster_gender_avg = defaultdict(Counter) 
for key in cluster_gender_sum: 
    for pole in cluster_gender_sum[key]: 
        if key.endswith('fem'): 
            pole_avg = np.mean(fem_scores[pole])
        elif key.endswith('masc'): 
            pole_avg = np.mean(masc_scores[pole])
        else: 
            continue
#         pole_avg = np.mean(scores[pole])
        cluster_gender_avg[key][pole] = cluster_gender_sum[key][pole] / cluster_gender_total[key][pole] - pole_avg

In [ ]:
cluster_order = [4, 3, 5, 1, 2, 0]
for clust_num in cluster_order: 
    for gender in ['fem', 'masc']:
        key = str(clust_num) + '_' + gender
        print(key)
        print(cluster_gender_avg[key].most_common()[:10])
        print(cluster_gender_avg[key].most_common()[-10:])
        print()